In [ ]:
pip install wandb

  Using cached wandb-0.16.6-py3-none-any.whl (2.2 MB)
  Using cached sentry_sdk-1.45.0-py2.py3-none-any.whl (267 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)


In [ ]:
pip install sigopt

In [ ]:
!pip install 'ray[tune]'

In [ ]:
!pip install optuna

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install datasets

In [ ]:
import os
import torch
import warnings
import datasets
import numpy as np
import pandas as pd
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings(action='ignore')

In [ ]:
model_checkpoint = 'klue/bert-base'
batch_size = 32
task = 'nli'
RANDOM_SEED = 17

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
dataset = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [ ]:
dataset_train, dataset_val = train_test_split(dataset, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_key, label_key, bert_tokenizer):
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=False) for i in dataset[sent_key]]
    if not label_key == None:
      self.mode = 'train'
    else:
      self.mode = 'test'
    if self.mode == 'train':
      self.labels = [np.int64(i) for i in dataset[label_key]]
    else:
      self.labels = [np.int64(0) for i in dataset[sent_key]]

  def __getitem__(self, i):
    if self.mode == 'train':
      self.sentences[i]['label'] = self.labels[i]
      return self.sentences[i]
    else:
      return self.sentences[i]

  def __len__(self):
    return (len(self.labels))

In [ ]:
data_train = BERTDataset(dataset_train, 'title', 'topic_idx', tokenizer)
data_val = BERTDataset(dataset_val, 'title', 'topic_idx', tokenizer)
data_test = BERTDataset(test, 'title', None, tokenizer)

In [ ]:
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = load_metric('glue', 'qnli')

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
num_labels = 7

def model_init():
  return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_run = trainer.hyperparameter_search(n_trials=2, direction='maximize')

[I 2024-04-23 12:26:45,798] A new study created in memory with name: no-name-2c5b1235-2443-4817-90e4-27571958b3eb
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train/epoch,▁▂▃▄▄▅▆▇█
train/global_step,▁▂▃▄▅▅▆▇█
train/grad_norm,▄▆█▂▁▄▄▁▂
train/learning_rate,█▇▆▅▅▄▃▂▁
train/loss,█▅▄▂▄▂▂▂▁
train/epoch,0.98555
train/global_step,4500
train/grad_norm,6.38429
train/learning_rate,4e-05
train/loss,0.4616


wandb: Currently logged in as: byeolki7007. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.357800,0.342205,0.888950


[I 2024-04-23 12:30:18,285] Trial 0 finished with value: 0.8889497316832767 and parameters: {'learning_rate': 1.0600246581460117e-05, 'num_train_epochs': 1, 'seed': 21, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.8889497316832767.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▆██
train/global_step,▁▆██
train/grad_norm,▁█
train/learning_rate,█▁
train/loss,█▁
eval/accuracy,0.88895


Epoch,Training Loss,Validation Loss,Accuracy
1,0.426500,0.447720,0.873946
2,0.319600,0.477958,0.879750
3,0.209600,0.590366,0.879531


[W 2024-04-23 12:55:31,218] Trial 1 failed with parameters: {'learning_rate': 3.743785246603542e-05, 'num_train_epochs': 4, 'seed': 10, 'per_device_train_batch_size': 8} because of the following error: SafetensorError('Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 200, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1859, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2278, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
best_run

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3c8c0f3730>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

NameError: name 'best_run' is not defined

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3c8c0f3730>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe